In [1]:
from enum import Enum


import inspect
from copy import deepcopy, copy
from inspect import Parameter, Signature
from types import FunctionType
from typing import *
from functools import lru_cache
from collections.abc import Mapping

from mousse import Dataclass, asdict, asclass
from mousse.types import get_fields_info, Accessor, Field, parse, parser

In [2]:
class Foo(Dataclass, strict=100):
    name: str
        
class Bar(Foo):
    age: int

In [5]:
a = Foo(name=1)
# a.name = Ellipsis

AssertionError: Invalid datatype: require <class 'str'>, get <class 'int'>

In [5]:
a.name = 1

<class 'int'> <class 'str'>


AssertionError: Invalid datatype: require <class 'str'>, get <class 'int'>

In [1]:
class ReadOnlyFieldException(Exception):
    def __init__(self, key: str):
        super().__init__(f"Field `{key}` is readonly")
        
        
class ConfigMetadata(Dataclass, dynamic=True):
    readonly: bool = False
        
        
class ConfigAccessor(Accessor):
    def __set__(self, obj: Any, val: Any):
        metadata = _get_metadata(obj, self.key)

        if metadata.readonly:
            raise ReadOnlyFieldException(self.key)

        metadata.readonly = True
        self.val = parse(Config, val)
        
    def __get__(self, obj: Any):
        return self.val
        
        
        
class Config(Dataclass, dynamic=True, accessor=ConfigAccessor):
    pass


        
@parser(Config)
def parse_config(G: Type[Config], config: Union[Mapping, list, tuple, set]):
    if isinstance(config, (list, tuple, set)):
        return tuple(parse_config(G, elem) for elem in config)
    
    if isinstance(config, Mapping):
        data = {}
        for key, val in config.items():
            val = parse_config(Config, val)
            data[key] = val
        
        return Config(**data)
    
    return config


@lru_cache(maxsize=None)
def _get_metadata(obj: Any, key: str) -> ConfigMetadata:
    return ConfigMetadata()


def update(config: Config, **kwargs):
    for key, val in kwargs.items():
        setattr(config, key, val)



NameError: name 'Dataclass' is not defined

In [26]:
config = Config()
update(config, name="datnh21", items=[[{"aa": "banana"}]])

In [27]:
for key in config:
    print(key)

('name', 'datnh21')
('items', ((Config(aa="banana"),),))


In [28]:
config_meta = ConfigMetadata()

In [30]:
config_meta.aw = "qq"

In [32]:
asdict(config_meta)

{'readonly': False, 'aw': 'qq'}

In [11]:
type(Mapping)

abc.ABCMeta

In [11]:
b

Bar(foo_lst=[Foo(age=10, name="thu")], foo_dct={'a': Foo(age=11, name="ad"), 'b': Foo(age=12, name="fd")}, foo_nst=[{'a': Foo(age=11, name="ad"), 'b': Foo(age=12, name="fd")}], foo=Foo(age=14, name="dat"))

In [12]:
asdict(b)

{'foo_lst': [{'age': 10, 'name': 'thu'}],
 'foo_dct': {'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}},
 'foo_nst': [{'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}}],
 'foo': {'age': 14, 'name': 'dat'}}